In [33]:
%matplotlib inline 

from utils import *

In [42]:
wk_dir = os.getcwd()

In [43]:
data_dir = wk_dir + "/../data/kg/cd-redux/sample/"

In [44]:
model_dir = data_dir + "/models/"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

### Finetuning from scratch(?)

In [45]:
vgg = Vgg16()
model = vgg.model

In [46]:
batches = get_batches(data_dir + "/train/", shuffle=False, batch_size=1)
val_batches = get_batches(data_dir + "/valid/", shuffle=False, batch_size=1)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


Using a batch_size of 1 doesn't mean we're not getting all the images - we are.

**What is batch_size?**

Number of images to process at once. We try to process as many images at a time as our graphics card allows.

Apparently when we use a batch_size of 1 it's because we're doing preprocessing on the CPU (??)

In [47]:
import bcolz
def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode="w")
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [48]:
val_data = get_data(data_dir + "/valid/")

Found 50 images belonging to 2 classes.


In [49]:
trn_data = get_data(data_dir + "/train/")

Found 200 images belonging to 2 classes.


```
def get_data(path, target_size=(224,224)):
    batches = get_batches(
        path, 
        shuffle=False, 
        batch_size=1, 
        class_mode=None, 
        target_size=target_size
    )
    return np.concatenate(
        [batches.next() for i in range(batches.nb_sample)]
    )
```

In [50]:
val_data[:1]

array([[[[ 153.,  153.,  154., ...,  157.,  159.,  162.],
         [ 153.,  153.,  154., ...,  157.,  159.,  162.],
         [ 149.,  149.,  150., ...,  151.,  153.,  155.],
         ..., 
         [ 251.,  251.,  252., ...,  251.,  249.,  248.],
         [ 251.,  251.,  252., ...,  251.,  249.,  248.],
         [ 251.,  251.,  252., ...,  251.,  249.,  248.]],

        [[ 193.,  193.,  194., ...,  198.,  200.,  203.],
         [ 193.,  193.,  194., ...,  198.,  200.,  203.],
         [ 189.,  189.,  190., ...,  192.,  194.,  196.],
         ..., 
         [ 255.,  255.,  255., ...,  248.,  246.,  245.],
         [ 255.,  255.,  255., ...,  248.,  246.,  245.],
         [ 255.,  255.,  255., ...,  248.,  246.,  245.]],

        [[ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 251.,  251.,  252., ...,  255.,  255.,  255.],
         ..., 
         [ 255.,  255.,  255., ...,  255.,  253.,  252.],
         [ 255.,  255.,

In [51]:
trn_data[:1]

array([[[[ 112.,  119.,  128., ...,  128.,  127.,  127.],
         [ 119.,  128.,  136., ...,  133.,  127.,  127.],
         [ 127.,  136.,  144., ...,  141.,  127.,  127.],
         ..., 
         [ 123.,  128.,  132., ...,  136.,  127.,  127.],
         [ 120.,  125.,  132., ...,  135.,  127.,  127.],
         [ 127.,  127.,  127., ...,  127.,  127.,  127.]],

        [[   3.,    2.,    3., ...,    6.,    0.,    0.],
         [   2.,    0.,    1., ...,    2.,    0.,    0.],
         [   0.,    0.,    0., ...,    0.,    0.,    0.],
         ..., 
         [   4.,    2.,    0., ...,    0.,    0.,    0.],
         [   4.,    2.,    0., ...,    0.,    0.,    0.],
         [   0.,    0.,    0., ...,    0.,    0.,    0.]],

        [[ 255.,  255.,  245., ...,  241.,  255.,  255.],
         [ 255.,  255.,  251., ...,  244.,  255.,  255.],
         [ 255.,  255.,  255., ...,  253.,  255.,  255.],
         ..., 
         [ 254.,  252.,  249., ...,  255.,  255.,  255.],
         [ 255.,  255.,

In [52]:
val_data.shape

(50, 3, 224, 224)

In [53]:
save_array(model_dir + "train_data.bc", trn_data)
save_array(model_dir + "valid_data.bc", val_data)

Why is this necessary? The notes say "loading and resizing the images every time we want to use them isn't necessary" - instead we should save the processed arrays. What processing was done though?

Seems like all we did up to this point was get_batches and concatenate.

In [54]:
def onehot(x):
    return np.array(
        OneHotEncoder().fit_transform(
            x.reshape(-1,1)
        ).todense()
    )

In [55]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

We're formatting the cats and dogs data into a one hot encoded format that we can use for the linear model (I guess).

In [60]:
trn_features = model.predict(trn_data, batch_size=100)
val_features = model.predict(val_data, batch_size=100)

###### Original classes (from data)
```
batches = get_batches(data_dir + "/train/", shuffle=False, batch_size=1)
trn_classes = batches.classes
```

In [67]:
trn_classes[0]

0

###### One hot encoded labels for our linear model

In [68]:
trn_labels[0]

array([ 1.,  0.])

###### Predictions from our VGG model using ImageNet classifications

In [69]:
trn_features[0][:10]

array([  3.3189e-08,   3.3747e-06,   2.3458e-06,   1.8934e-06,   2.3422e-07,   8.1195e-06,
         9.3792e-08,   3.8643e-07,   1.2771e-07,   6.5615e-08], dtype=float32)

In [70]:
trn_features.shape

(200, 1000)

Note that every prediction contains 1,000 probabilities

In [71]:
save_array(model_dir + "train_lastlayer_features.bc", trn_features)
save_array(model_dir + "valid_lastlayer_features.bc", val_features)

Saving the ImageNet class predictions so we don't have to run it again later.

In [72]:
lm = Sequential(
    [Dense(2, input_shape=(1000,), activation="softmax")]
)
lm.compile(
    optimizer=RMSprop(lr=0.1), 
    loss="categorical_crossentropy"
)

You actually know this one! A linear model with a single NN layer that takes an input (1,000 ImageNet predictions) and produces an output (1 or 0 for cat or dog).

In [75]:
lm.fit(
    trn_features,
    trn_labels,
    nb_epoch=3,
    batch_size=64,
    validation_data=(
        val_features,
        val_labels
    )
)

Train on 200 samples, validate on 50 samples
Epoch 1/3
200/200 [==============================] - 0s - loss: 0.2756 - val_loss: 0.2695
Epoch 2/3
200/200 [==============================] - 0s - loss: 0.2057 - val_loss: 0.2317
Epoch 3/3
200/200 [==============================] - 0s - loss: 0.1647 - val_loss: 0.2080


In [76]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_7 (Dense)                  (None, 2)             2002        dense_input_1[0][0]              
Total params: 2002
____________________________________________________________________________________________________


The model has a single dense layer - not sure what "params" means here or "connected to"